## 1. Setup and Initialization
Before diving into use cases, we should initialize the notebook with the required setup:

In [1]:
# Import the helper script
import helpers.hopenai as hopenai

# Set up logging for debugging
import logging
logging.basicConfig(level=logging.INFO)

# Set OpenAI API key
import os
from typing import List

In [2]:
os.environ["OPENAI_API_KEY"] = "<your_openai_api_key>"

## 1. Travel Agent chat assistant: 
#### Goal: Cretae a chat agent that will help the user to create an itinary to visit New York Trip considering all the constraints.

In [3]:
# Define the prompt for the travel assistant
user_prompt = """
I am visiting New York City for 3 days. Please create a detailed itinerary, 
including popular attractions, food recommendations, and some evening activities.
I already booked flight tickets and hotel near Newark penn station.
Constraints:
1) Dates: from 24th to 27th Dec.
1) My budget for travel is around $400 excluding hotel and flight.
2) I am planning to travel through subway and for rest of the trip I am planning to walk.
3) Also, take into account traffic and tourist rush at popular places.
"""

# Define the system instructions for the assistant
system_instructions = """
You are a travel assistant specializing in creating personalized travel itineraries.
Your recommendations should balance sightseeing, food, and leisure activities considering provided constraints.
Provide details like the time required for activities and approximate costs where possible.
"""

# Use the get_completion method to generate the trip plan
trip_plan = hopenai.get_completion(
    user=user_prompt,
    system=system_instructions,
    model="gpt-4o-mini",
    temperature=0.7  # Slightly increase temperature for creative outputs
)

# Print the generated trip itinerary
print("3-Day New York City Trip Itinerary:")
print(trip_plan)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


3-Day New York City Trip Itinerary:
Here’s a detailed 3-day itinerary for your trip to New York City from December 24th to 27th. This itinerary balances sightseeing, food, and leisure activities while considering your budget and travel preferences.

### Day 1: December 24th (Christmas Eve)

**Morning:**
- **Breakfast:** Start your day at **Ess-a-Bagel** (approx. $5-10). Enjoy a classic New York bagel with cream cheese.
- **Visit Central Park:** Take the subway to **59th St – Columbus Circle** (approx. $2.75). Walk around the park, see the winter scenery, and visit the **Bethesda Terrace**. (2-3 hours)

**Afternoon:**
- **Lunch:** Head to **The Halal Guys** (approx. $10) for a famous street food experience. 
- **Visit the Museum of Modern Art (MoMA):** Take the subway from **59th St to 53rd St** (approx. $2.75). Admission is $25. Spend around 2-3 hours exploring the exhibits.

**Evening:**
- **Dinner:** Try **Joe's Pizza** in Greenwich Village (approx. $3-5 per slice).
- **Evening Activ

## 2. Chatbot for Coding Assistance
#### Goal: Create a chatbot that assists developers with coding questions based on the provided coding style guide.

In [4]:
# Define the assistant name and instructions
assistant_name = "CodingAssistant"
instructions = "You are a helpful coding assistant. Answer technical questions clearly and concisely."
vector_store_name = "coding_help_vector_store"

# Provide relevant documentation files
file_paths = ["../helpers_root/docs/coding/all.coding_style.how_to_guide.md"]

# Create or retrieve the assistant
assistant = hopenai.get_coding_style_assistant(
    assistant_name=assistant_name,
    instructions=instructions,
    vector_store_name=vector_store_name,
    file_paths=file_paths
)

# Query the assistant
question = "What are common python mistaks that I should keep in mind while writing code?"
response_messages = hopenai.get_query_assistant(assistant, question)

# Display the assistant's response
for message in response_messages:
    # Ensure the message has content to process
    if hasattr(message, "content"):
        for content_block in message.content:
            if hasattr(content_block, "text") and hasattr(content_block.text, "value"):
                print(f"{message.role}: {content_block.text.value}")
            else:
                print(f"{message.role}: [No valid text content found]")
    else:
        print("[No content attribute in the message]")



INFO:httpx:HTTP Request: GET https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Creating a new assistant: CodingAssistant
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Vector store 'coding_help_vector_store' already exists. Using it
INFO:helpers.hopenai:Uploading files to vector store ...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores/vs_uwnZJE3zIWUdlGNh1T6NVFcp/file_batches "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_uwnZJE3zIWUdlGNh1T6NVFcp/file_batches/vsfb_1d0e9241d2194446a0ac07f22e07cb1a "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_uwnZJE3zIWUdlGNh1T6NVFcp/file_batches/vsfb_1d0e9241d2194446a0ac07f22e07cb1a "HTTP/1.1 200 O

assistant: Here are some common mistakes to avoid when writing Python code:

1. **Indentation Errors**: Python relies on indentation to define the structure of the code. Ensure consistent use of spaces or tabs.

2. **Misusing Mutable Default Arguments**: Using mutable default arguments (like lists or dictionaries) can lead to unexpected behavior because they are shared across function calls. Instead, use `None` and initialize inside the function.

   ```python
   def my_function(arg=None):
       if arg is None:
           arg = []
   ```

3. **Not Handling Exceptions**: Failing to handle exceptions can crash your program. Use try-except blocks to catch and handle exceptions gracefully.

   ```python
   try:
       # some code that may raise an exception
   except SomeException as e:
       print(e)
   ```

4. **Overusing Global Variables**: Relying heavily on global variables can lead to code that is hard to debug and maintain. Instead, prefer passing parameters to functions.

5. **Us

## 3. Managing Uploaded Files
#### Goal: List, view, and delete files in the OpenAI API.

In [5]:
def print_file_info(files_list: List[str]) -> None:
    """
    Prints file info.
    """
    for file in files_list:
        file_info = hopenai.file_to_info(file)
        print(file_info)

In [6]:
# List all files
client = hopenai.OpenAI()
files_before = list(client.files.list())
print("Uploaded files:")
print_file_info(files_before)
# Delete all files (with confirmation)
hopenai.delete_all_files(ask_for_confirmation=False)
# Verify deletion
files_after = list(client.files.list())
print("Files after deletion:")
print_file_info(files_after)


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/files?after=file-VEzEvpmVfhgUTpH5dmWtxC "HTTP/1.1 200 OK"


Uploaded files:
{'id': 'file-QFUuXpobjoLCnLBaCX2HK5', 'created_at': datetime.datetime(2024, 12, 11, 20, 14), 'filename': 'all.coding_style.how_to_guide.md'}
{'id': 'file-5m1CX8UeABBJtFATb3cLtz', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.write_unit_tests.how_to_guide.md'}
{'id': 'file-UhqL4yhYrFP8ziDmM3XzB1', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.coding_style.how_to_guide.md'}
{'id': 'file-VEzEvpmVfhgUTpH5dmWtxC', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.imports_and_packages.how_to_guide.md'}


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/files?after=file-VEzEvpmVfhgUTpH5dmWtxC "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Found 4 files
Deleting file {'id': 'file-QFUuXpobjoLCnLBaCX2HK5', 'created_at': datetime.datetime(2024, 12, 11, 20, 14), 'filename': 'all.coding_style.how_to_guide.md'}
Deleting file {'id': 'file-5m1CX8UeABBJtFATb3cLtz', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.write_unit_tests.how_to_guide.md'}
Deleting file {'id': 'file-UhqL4yhYrFP8ziDmM3XzB1', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.coding_style.how_to_guide.md'}
Deleting file {'id': 'file-VEzEvpmVfhgUTpH5dmWtxC', 'created_at': datetime.datetime(2024, 12, 11, 19, 59, 4), 'filename': 'all.imports_and_packages.how_to_guide.md'}
INFO:helpers.hopenai:Deleting file FileObject(id='file-QFUuXpobjoLCnLBaCX2HK5', bytes=90231, created_at=1733948040, filename='all.co

Files after deletion:


## 4. Batch Upload to Vector Store
#### Goal: Add multiple files to a vector store and check their status.

In [7]:
# Upload files to a vector store
vector_store_name = "batch_vector_store"
file_paths = ["../helpers_root/docs/coding/all.imports_and_packages.how_to_guide.md", "../helpers_root/docs/coding/all.write_unit_tests.how_to_guide.md",
             "../helpers_root/docs/coding/all.coding_style.how_to_guide.md"]  # Example paths

# Create or find vector store
client = hopenai.OpenAI()
vector_store = client.beta.vector_stores.create(name=vector_store_name)

# Upload files to the vector store
file_streams = [open(path, "rb") for path in file_paths]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Display file batch status
print(f"File batch status: {file_batch.status}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores/vs_HvRISC4fiissfWRgMkxEYZvv/file_batches "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_HvRISC4fiissfWRgMkxEYZvv/file_batches/vsfb_d014a267706a429c85aa224988925ba0 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_HvRISC4fiissfWRgMkxEYZvv/file_batches/vsfb_d014a267706a429c85aa224988925ba0 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_HvRISC4fiissfWRgMkxEYZvv/file_batches/vsfb_d014a267706a429c85aa224988925ba0 "HTTP/1.1 200 OK"


File batch status: completed


## 5. Tasking an AI Agent to Generate Code and Unit Tests Using Coding Guides
#### Scenario:

You are a developer working on a new feature for a Python application. You uploaded documentation about best practices for imports, writing unit tests, and coding styles to a vector store. Now, you want to create an AI agent that can generate boilerplate code and its corresponding unit tests for a given functionality while adhering to these guides.

In [8]:
# Step 1: Create a coding assistant
assistant_name = "CodeAndTestAssistant"
instructions = (
    "You are a coding assistant trained to write Python code and unit tests."
    "Adhere strictly to the following rules based on the uploaded guide:"
    "- Adhere to PEP 8 coding standards."
    "- Use proper imports as documented in the coding guides."
    "- Write comprehensive and edge-case-aware unit tests."
)

# Create or retrieve the assistant
coding_assistant = hopenai.get_coding_style_assistant(
    assistant_name=assistant_name,
    instructions=instructions,
    vector_store_name=vector_store_name,
    file_paths=None,  # Files already uploaded to the vector store
)

# Step 2: Query the assistant to generate code and tests
task = (
    "Write a Python function `calculate_area` that computes the area of a rectangle "
    "given its width and height. Then, write unit tests to verify its functionality."
)

response_messages = hopenai.get_query_assistant(coding_assistant, task)

# Display the assistant's response
for message in response_messages:
    # Ensure the message has content to process
    if hasattr(message, "content"):
        for content_block in message.content:
            if hasattr(content_block, "text") and hasattr(content_block.text, "value"):
                print(f"{message.role}: {content_block.text.value}")
            else:
                print(f"{message.role}: [No valid text content found]")
    else:
        print("[No content attribute in the message]")

INFO:httpx:HTTP Request: GET https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Creating a new assistant: CodeAndTestAssistant
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Vector store 'batch_vector_store' already exists. Using it
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants/asst_xVgsDQgFV8ZILTNWNuLSSUct "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
INFO:helpers.hopenai:thread=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_5mCof6nDc3wT12u3PGvc0dTS/runs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_5mCof6nDc3wT12u3PGvc0dTS/runs/run_OjzkBDRXP4QvMKnFyNqbw7xq "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_5mCof6nDc3wT12u3PGvc0dTS/runs/

assistant: Here's the Python function `calculate_area` that computes the area of a rectangle given its width and height, along with unit tests to verify its functionality.

### Python Function

```python
def calculate_area(width: float, height: float) -> float:
    """Calculate the area of a rectangle.

    Args:
        width (float): The width of the rectangle.
        height (float): The height of the rectangle.

    Returns:
        float: The area of the rectangle.

    Raises:
        ValueError: If width or height is negative.
    """
    if width < 0 or height < 0:
        raise ValueError("Width and height must be non-negative.")
    
    return width * height
```

### Unit Tests

Below are the unit tests for the `calculate_area` function using the `unittest` framework.

```python
import unittest

class TestCalculateArea(unittest.TestCase):
    
    def test_positive_dimensions(self):
        self.assertEqual(calculate_area(5, 10), 50)
        self.assertEqual(calculate_area(2

## 6. Manage assistants.
#### Goal: List and delete existing assistants.

In [9]:
def print_assistant_info(assistants_list: List[str]) -> None:
    """
    Prints assistant info.
    """
    for assistant in assistants_list:
        assistant_info = hopenai.assistant_to_info(assistant)
        print(assistant_info)
        
# List all assistants
assistants = client.beta.assistants.list()
print("Assistants:")
print_assistant_info(assistants.data)

# Delete all assistants (with confirmation)
hopenai.delete_all_assistants(ask_for_confirmation=False)

# Verify deletion
assistants_after = client.beta.assistants.list()
print("Assistants after deletion:")
print_assistant_info(assistants_after.data)


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"


Assistants:
{'name': 'CodeAndTestAssistant', 'created_at': datetime.datetime(2024, 12, 11, 20, 14, 26), 'id': 'asst_xVgsDQgFV8ZILTNWNuLSSUct', 'instructions': 'You are a coding assistant trained to write Python code and unit tests.Adhere strictly to the following rules based on the uploaded guide:- Adhere to PEP 8 coding standards.- Use proper imports as documented in the coding guides.- Write comprehensive and edge-case-aware unit tests.', 'model': 'gpt-4o-mini'}
{'name': 'CodingAssistant', 'created_at': datetime.datetime(2024, 12, 11, 20, 13, 59), 'id': 'asst_XMbluEVX6o7bdzA4iMIjBaJC', 'instructions': 'You are a helpful coding assistant. Answer technical questions clearly and concisely.', 'model': 'gpt-4o-mini'}


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:helpers.hopenai:Found 2 assistants
Deleting assistant {'name': 'CodeAndTestAssistant', 'created_at': datetime.datetime(2024, 12, 11, 20, 14, 26), 'id': 'asst_xVgsDQgFV8ZILTNWNuLSSUct', 'instructions': 'You are a coding assistant trained to write Python code and unit tests.Adhere strictly to the following rules based on the uploaded guide:- Adhere to PEP 8 coding standards.- Use proper imports as documented in the coding guides.- Write comprehensive and edge-case-aware unit tests.', 'model': 'gpt-4o-mini'}
Deleting assistant {'name': 'CodingAssistant', 'created_at': datetime.datetime(2024, 12, 11, 20, 13, 59), 'id': 'asst_XMbluEVX6o7bdzA4iMIjBaJC', 'instructions': 'You are a helpful coding assistant. Answer technical questions clearly and concisely.', 'model': 'gpt-4o-mini'}
INFO:helpers.hopenai:Deleting assistant Assistant(id='asst_xVgsDQgFV8ZILTNWNuLSSUct', created_at=1733948066, description=None,

Assistants after deletion:
